# Convert VTK to XDMF

In [ ]:
import pyvista as pv
import dolfinx as dlx
from mpi4py import MPI


patient_id = "patient_01"
infile_name = f"../data/{patient_id}/mesh_with_uacs_fibers_tags.vtk"
outfile_name = f"../data/{patient_id}/mesh_dolfinx.xdmf"
mpi_communicator = MPI.COMM_WORLD

vtk_mesh = pv.read(infile_name)
vtk_mesh.cell_data.clear()
vtk_mesh.point_data.clear()
pv.save_meshio(outfile_name, vtk_mesh)

dolfinx_mesh = dlx.io.XDMFFile(mpi_communicator, outfile_name, "r").read_mesh(name="Grid")

# Coarsen Mesh

In [5]:
import pyvista as pv


patient_id = "patient_01"
infile_name = f"../data/{patient_id}/mesh_with_uacs_fibers_tags.vtk"
outfile_name = f"../data/{patient_id}/mesh_coarse_with_uacs_tags.vtk"
decimation_fraction = 0.8

vtk_mesh = pv.read(infile_name)
points = vtk_mesh.points
simplices = vtk_mesh.cells.reshape(-1, 4)[:, 1:]
alpha_values = vtk_mesh.point_data["alpha"]
beta_values = vtk_mesh.point_data["beta"]
triangular_mesh = pv.PolyData.from_regular_faces(points, simplices)
triangular_mesh.point_data["alpha"] = alpha_values
triangular_mesh.point_data["beta"] = beta_values
coarse_mesh = triangular_mesh.decimate(
    decimation_fraction, enable_all_attribute_error=True, volume_preservation=False
)
pv.save_meshio(outfile_name, coarse_mesh)